# 01_data_collection.ipynb

**Project:** king_county_housing_data  
**Author:** Johannes Gooth  
**Date:** April 12, 2024

---

## Introduction

In this notebook, we will focus on the data collection phase of our analysis. The primary objective is to gather and compile the necessary dataset that will be used throughout the project. This involves acquiring data from a PostgreSQL database in a suitable format and performing an initial check to confirm its integrity.

The dataset we will be working with is the King County Housing Data. This dataset contains detailed information about home sales in King County, USA. The data is stored on an AWS cloud server managed by NeueFische GmbH. We have explored the dataset using DBeaver and have identified the necessary join between two tables to extract the relevant information.

### Key Steps:
1. **Connecting to the Data Source:** We have explored two methods to connect to PostgreSQL databases and export the data to a CSV file: using `psycopg2` and `SQLAlchemy`.  
   To establish this connection, we need the following information:
   - **host:** The address of the machine where the database is hosted.
   - **port:** The virtual gate number through which communication will be allowed.
   - **database:** The name of the database.
   - **user:** The username.
   - **password:** The user's password.

   Since we don't want to publish the database credentials on GitHub, we store them in a `.env` file, which is included in the `.gitignore`. This file allows us to securely store sensitive information that should not be made public. Using the `dotenv` package, we can read the `.env` file and access the variables.

2. **Data Acquisition:** We will collect the data from the identified source by querying the PostgreSQL database using appropriate tools and techniques.

3. **Initial Data Inspection:** Once the data is collected, we will perform a quick inspection to verify its structure and ensure it meets our requirements.

4. **Storage and Organization:** The collected data will be saved in a CSV file format, making it ready for further processing and analysis in subsequent notebooks.

### Expected Outcome:
A raw dataset, organized and stored in a CSV file, ready for further analysis.

## Setting-Up the Working Enviroment

In [85]:
import os
from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

## Connecting and Retrieving Data via `psycopg2`

### Load Environment Variables
We will load our database credentials from the .env file using the dotenv package. This ensures that sensitive information, such as the database password, is not exposed in our code.

In [86]:
load_dotenv()

DATABASE = os.getenv('DATABASE')
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

### Establish the Database Connection
Next, we will establish a connection to the PostgreSQL database using psycopg2 and create a cursor object to execute our SQL queries.

In [87]:
try:
    # Connect to your PostgreSQL database
    connection = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)
    cursor = connection.cursor()
    print("Database connection successful.")

except Exception as error:
    print(f"Error connecting to the database: {error}.")

Database connection successful.


### Execute SQL Queries and Fetch Data
With the connection established, we can now execute our SQL queries to retrieve the necessary data. For example, we join two tables and fetch the results.

In [88]:
query = """
SELECT d.*, s.date, s.price
FROM eda.king_county_house_sales AS s
INNER JOIN eda.king_county_house_details AS d
ON s.house_id = d.id
"""

cursor.execute(query)
data = cursor.fetchall()

### Convert Data to DataFrame
Once the data is retrieved, we will convert it into a pandas DataFrame for easier manipulation and analysis.

In [89]:
columns = [desc[0] for desc in cursor.description]
df_psycopg2 = pd.DataFrame(data, columns=columns)

### Initial Data Inspection
Quick inspection of the data to verify its structure and ensure it meets our requirements.

In [90]:
df_psycopg2.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price
0,7129300520,3.0,1.00,1180.0,5650.0,1.0,NaN,0.0,3,7,...,0.0,1955,0.0,98178,47.5112,-122.257,1340.0,5650.0,2014-10-13,221900.0
1,6414100192,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3,7,...,400.0,1951,19910.0,98125,47.7210,-122.319,1690.0,7639.0,2014-12-09,538000.0
2,5631500400,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3,6,...,0.0,1933,NaN,98028,47.7379,-122.233,2720.0,8062.0,2015-02-25,180000.0
3,2487200875,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5,7,...,910.0,1965,0.0,98136,47.5208,-122.393,1360.0,5000.0,2014-12-09,604000.0
4,1954400510,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3,8,...,0.0,1987,0.0,98074,47.6168,-122.045,1800.0,7503.0,2015-02-18,510000.0


The data has the desired structure and meets our requirements.

### Close the Connection
After the data is retrieved and saved, close the cursor and connection to the database.

In [91]:
cursor.close()
connection.close()
print("Database connection closed.")

Database connection closed.


## Connecting and Retrieving Data via `SQLAlchemy`

In this section, we will use `SQLAlchemy` to connect to our PostgreSQL database and retrieve the necessary data. SQLAlchemy provides a high-level ORM (Object Relational Mapper) that simplifies database interactions. Here’s how we can do it:

### Load Environment Variables
Similar to how we handled it with `psycopg2`, we will load our database credentials from the .env file using the dotenv package.

In [92]:
load_dotenv()

DATABASE = os.getenv('DATABASE')
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')


### Create a Database Engine
With `SQLAlchemy`, we create an engine using a database connection string. This string encapsulates all the information needed to connect to the database.

In [93]:
# Create the database connection string
db_string = f"postgresql://{USER_DB}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"

# Create the SQLAlchemy engine
engine = create_engine(db_string)

### Connect to the Database and Execute SQL Queries
With the engine created, we can now connect to the database, execute SQL queries, and import the data into a pandas dataframe.

In [94]:
query = """
SELECT d.*, s.date, s.price
FROM eda.king_county_house_sales AS s
INNER JOIN eda.king_county_house_details AS d
ON s.house_id = d.id
"""

df_sqlalchemy = pd.read_sql(query, engine)


### Initial Data Inspection
Quick inspection of the data to verify its structure and ensure it meets our requirements.

In [95]:
df_sqlalchemy.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price
0,7129300520,3.0,1.00,1180.0,5650.0,1.0,NaN,0.0,3,7,...,0.0,1955,0.0,98178,47.5112,-122.257,1340.0,5650.0,2014-10-13,221900.0
1,6414100192,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3,7,...,400.0,1951,19910.0,98125,47.7210,-122.319,1690.0,7639.0,2014-12-09,538000.0
2,5631500400,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3,6,...,0.0,1933,NaN,98028,47.7379,-122.233,2720.0,8062.0,2015-02-25,180000.0
3,2487200875,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5,7,...,910.0,1965,0.0,98136,47.5208,-122.393,1360.0,5000.0,2014-12-09,604000.0
4,1954400510,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3,8,...,0.0,1987,0.0,98074,47.6168,-122.045,1800.0,7503.0,2015-02-18,510000.0


The data has the desired structure and meets our requirements.

### Close the Engine Connection
`SQLAlchemy` engines don’t require explicit closing like `psycopg2` connections, but it’s a good practice to ensure that all connections are properly managed. Therefore, we explicitly close the engine:

In [96]:
engine.dispose()

## Closing

### Verify DataFrame Equality
To ensure the consistency of data retrieved using both `psycopg2` and `SQLAlchemy`, we will compare the two DataFrames (df_psycopg2 and df_sqlalchemy). This comparison will check if the DataFrames are identical in terms of shape and data content.

In [97]:
# Check if the DataFrames have the same shape
if df_psycopg2.shape != df_sqlalchemy.shape:
    print("The DataFrames have different shapes, so they are not the same.")
else:
    # Check if all the elements in the DataFrames are the same
    if df_psycopg2.equals(df_sqlalchemy):
        print("The DataFrames are identical.")
    else:
        print("The DataFrames are not identical. There are differences in the data.")


The DataFrames are identical.


Great! The comparison shows that the data retrieved via psycopg2 and SQLAlchemy is identical. This confirms the consistency of our data extraction methods. We can now confidently move on to the next steps: data cleaning and preprocessing.

### Save the Data to a CSV File
The data is retrieved and verified. Now, it is saveed to a CSV file for further analysis.

In [98]:
df_sqlalchemy.to_csv("../data/king_county_housing_data.csv", index=False)